# Database Connection

In [41]:
import pandas as pd
import numpy as np
import redshift_connector
conn = redshift_connector.connect(
     host='HostName',
     database='DataBase Name',
     port=0000,
     user='User',
     password='Password'
  )
pd.set_option('display.max_columns', None)
cursor = conn.cursor()

# Bucketing the Rule Set

In [ ]:
query = """
SELECT r.rulename, i.severity
FROM infosec_cdap_log_extschema.epo_new_udlp_incidents i
JOIN infosec_cdap_log_extschema.epo_new_udlp_incidentrules r
ON i.incidentid = r.incidentid
GROUP BY r.rulename, i.severity
ORDER BY r.rulename, i.severity;
"""
rules = pd.read_sql_query(query, conn)

In [56]:
list_rules = set(rules['rulename'].tolist())

buckets = {
    "Print Control": [
        'PP - 01 -  Print Control', 'PP - 01 -  Print Control', 
        'PP - 01 -  Print Control', 'PP  - 01 -  Print Control', 'Print Control', 'Printer Protection'
    ],
    "Device Monitoring and Control": [
        '[9.3]  - McAfee Device Control Policy - Monitor', 
        '[9.3]  - McAfee Device Control Policy - Monitor File Copy', 
        '[9.3]  - McAfee Device Control Policy - Monitor File Copy', 
        '[9.3]  - McAfee Device Control Policy - Monitor File Copy', 
        '[9.3.1]  - McAfee Device Control Policy - Monitor File Copy (Incoming)',
         '[9.3]  - McAfee Device Control Policy - Block', 
        '[9.3]  - McAfee Device Control Policy - Read Only', 
        '[9.3]  - Mobile Device Control - Block', 
        '[9.3]  - Mobile Device Control - Monitor',
       
    ],
    "Plug in Data Transfer": [
        ' - Plug and Play - USB - Block', 
        ' - Plug and Play - USB - Monitor',
        '! USB Printer', 'Removable Storage', 
        ' - Plug and Play - WPD - Block', 
        ' - Plug and Play - WPD - Monitor',
        'BT File Transfers', 'Bluetooth RFCOMM',  
        ' - Plug and Play - USB - Monitor', 
        ' - Plug and Play - WPD - Monitor'
    ],

    
     "Post": [
         
        'WP - 01 - PII Post', 'WP - 01b - PII Post', 'WP - 01b - PII Post', 
        'WP - 01d - PII Post', 'WP - 01f - PII Post', 'WP - 02 - PCI Post', 
        'WP - 02b - PCI Post', 'WP - 02b - PCI Post', 'WP - 02d - PCI Post'
    ],
    
    "Upload": [
         'WP - 01a - PII Upload', 
        'WP - 01c - PII Upload', 'WP - 01c - PII Upload', 'WP - 01e - PII Upload', 
        'WP - 01e - PII Upload', 'WP - 01g - PII Upload', 'WP - 02a - PCI Upload', 
        'WP - 02c - PCI Upload', 'WP - 02c - PCI Upload', 'WP - 02e - PCI Upload', 
        'WP - 03 - SC Upload', 'WP - 04 - SD Upload', 'WP - 04 - SD Upload'
    ],
    
    "Uncategorized": ['EP - 01b - PII - PD', 'PP - 02 - PII', 'RSP - 01 - PII', 
                      'EP - 02b - PCI - PD', 'EP - 04 - SD - PD', 'PP - 02a - PII', 
                    'PP - 02a - PII', 'PP - 03 - PCI', 'PP - 03a - PCI', 'PP - 03a - PCI', 
                    'RSP - 01a - PII', 'RSP - 02a - PCI', 'RSP - 02b - PCI', 'RSP - 02b - PCI', 
                    'RSP - 03 - SC', 'RSP - 04 - SD', 'RSP - 04 - SD']
}

## Bucketing Cases

In [ ]:
query = """
SELECT i.incidentid, i.severity, i.violationtimezone, trunc(i.violationutctime) as violation_time, i.computerid, i.userid, COALESCE(split_part(i.userprincipalname,'@',1),
split_part(i.useraccount, '\\\\', 2)) as username, r.evidenceid, r.rulename, e.filesize as filesize_kb, e.filename
FROM _infosec_cdap_log_extschema.epo_new_udlp_incidents i
JOIN _infosec_cdap_log_extschema.epo_new_udlp_incidentrules r ON i.incidentid = r.incidentid
JOIN _infosec_cdap_log_extschema.epo_new_udlp_incidentevidences e ON r.evidenceid = e.evidenceid
limit 200000;
"""

epo_ = pd.read_sql_query(query, conn)

In [45]:
epo_.shape
epo_df = epo_.copy()
epo_df[['incidentid', 'severity', 'violationtimezone', 'violation_time',
        'username', 'evidenceid',
       'filesize_kb', 'filename']]

,incidentid,severity,violationtimezone,violation_time,username,evidenceid,filesize_kb,filename
0,Anonymous,1,China Standard Time,2023-11-23,Anonymous,Anonymous,0,or.do.txt
1,Anonymous,1,China Standard Time,2023-11-26,Anonymous,Anonymous,33,tayo 10 profiles.docx
2,Anonymous,1,China Standard Time,2023-11-26,Anonymous,Anonymous,50,greenbulb agency contract 2013.docx01
3,Anonymous,1,China Standard Time,2023-11-26,Anonymous,Anonymous,22,myko news article- ahmet.docx
4,Anonymous,1,China Standard Time,2023-11-26,Anonymous,Anonymous,1,ncr opening-myko.docx01
...,...,...,...,...,...,...,...,...
Anonymous,Anonymous,1,China Standard Time,2024-04-18,pu52,Anonymous,94,WcmJavaViewer.jsp.pdf.txt
Anonymous,Anonymous,1,China Standard Time,2024-04-18,Anonymous,Anonymous,6410,getContent.doc
Anonymous,Anonymous,1,China Standard Time,2024-04-18,Anonymous,Anonymous,294,Print snip.txt
Anonymous,Anonymous,1,Pacific Daylight Time,2024-04-18,Anonymous,Anonymous,1,Advisor record of client conversation.pdf.txt


In [44]:
epo_df.columns

Index(['incidentid', 'severity', 'violationtimezone', 'violation_time',
       'computerid', 'userid', 'username', 'evidenceid', 'rulename',
       'filesize_kb', 'filename'],
      dtype='object')

## Map Rules to Dictionary

In [ ]:
reverse_buckets = {}
for main_category, subcategories in buckets.items():
    for subcategory in subcategories:
        reverse_buckets[subcategory] = main_category
epo_df['rulename'] = epo_df['rulename'].map(reverse_buckets)
epo_df['rulename'] = epo_df['rulename'].fillna('Uncategorized') 

In [ ]:
categorized_rules = {}
unique_rulenames = epo_['rulename'].unique()
for rule in unique_rulenames:
    for category, rules in buckets.items():
        if rule in rules:
            categorized_rules.setdefault(category, []).append(rule)
            break
    else:
        categorized_rules.setdefault('Uncategorized', []).append(rule)

### Data Info and Rule Severity Correspondence

In [169]:
epo_df.head()

,incidentid,severity,violationtimezone,violation_time,computerid,userid,username,evidenceid,rulename,filesize_kb,filename
0,Anonymous,1,China Standard Time,2023-10-27,Anonymous,Anonymous,Anonymous,Anonymous,Print Control,58,lbc transmittal 2 (Autosaved).xls
1,Anonymous,2,China Standard Time,2023-10-27,Anonymous,Anonymous,Anonymous,Anonymous,Post,205,HTTP Request Payload.txt
2,Anonymous,1,China Standard Time,2023-10-27,Anonymous,Anonymous,Anonymous,Anonymous,Print Control,1,Wong Chung Fai滿 65歲供款REV.PDF.txt
3,Anonymous,1,China Standard Time,2023-10-27,Anonymous,Anonymous,Anonymous,Anonymous,Print Control,6,GeneratePDF.pdf.txt
4,Anonymous,2,China Standard Time,2023-10-27,Anonymous,Anonymous,Anonymous,Anonymous,Post,205,HTTP Request Payload.txt


In [188]:
print(f"There are {epo_df['username'].nunique()} unique usernames")
print('-----------------------\n')
print(f"Rules have been bucketed into one of the following categories: {', '.join(epo_df['rulename'].unique())}")
print('-----------------------\n')
print(f"Violation Times range from {epo_df['violation_time'].min()} to {epo_df['violation_time'].max()}")
print('-----------------------\n')
print(f"The Shape of the Dataframe: {epo_df.shape}")

There are 8057 unique usernames
-----------------------

Rules have been bucketed into one of the following categories: Print Control, Post, Upload, Device Monitoring and Control, Uncategorized, Plug in Data Transfer
-----------------------

Violation Times range from 2023-08-23 to 2024-03-27
-----------------------

The Shape of the Dataframe: (200000, 11)


In [163]:
epo_df.isna().sum()

incidentid           0
severity             0
violationtimezone    0
violation_time       0
computerid           1
userid               0
username             0
evidenceid           0
rulename             0
filesize_kb          0
filename             0
dtype: int64

In [68]:
import matplotlib.pyplot as plt
plt.scatter(epo_df['severity'], epo_df['rulename'])
plt.xlabel('Severity Level')
plt.ylabel('Rule Name')
plt.xticks([1, 2, 3, 4])
plt.title('Severity Levels for Rule Buckets')
plt.show()

# Aggregating the Data

In [ ]:
full = epo_df.copy()
user_stats = full.groupby('username').agg({
    'violation_time': 'nunique',
    'rulename': 'nunique'
})

user_stats.rename(columns={
    'violation_time': 'Unique Violation Days',
    'rulename': 'Unique Rule Violations'
}, inplace=True)

user_stats.reset_index(inplace=True)

user_stats.rename(columns={'username': 'user'}, inplace=True)

fullc = user_stats[['Unique Violation Days', 'Unique Rule Violations', 'user']]

fullc['Total Violations'] = None

fullc['Unique Rule Violations'] = fullc['Unique Rule Violations'].astype('int')
sev_counts = full.groupby(['username', 'severity']).size().unstack(fill_value=0)
sev_counts.columns = ['severity_1', 'severity_2', 'severity_3', 'severity_4']
fullc = fullc.join(sev_counts, on='user',  rsuffix='_count')

print_counts = full.groupby(['username', 'rulename']).size().unstack(fill_value=0)
file_sizes = full.groupby(['username', 'rulename'])['filesize_kb'].sum().unstack(fill_value=0)

print_counts = print_counts.join(file_sizes, rsuffix='_file_size')
fullc = fullc.join(print_counts, on='user',  rsuffix='_count')
fullc['Total Violations'] = 0
fullc['Total Violations'] = fullc['severity_1'] + fullc['severity_2'] + fullc['severity_3'] + fullc['severity_4']
file_size_sum = full.groupby(['username', 'severity'])['filesize_kb'].sum().unstack(fill_value=0)

fullc = fullc.join(file_size_sum, on='user')
fullc.rename(columns={1: 'severity1_file_size', 2: 'severity2_file_size', 3: 'severity3_file_size', 4: 'severity4_file_size'}, inplace=True)
fullc['Sum File Size'] = fullc['severity1_file_size'] + fullc['severity2_file_size'] + fullc['severity3_file_size'] + fullc['severity4_file_size']
cols = ['user', 'Unique Violation Days', 'Unique Rule Violations', 
       'Total Violations', 'severity_1', 'severity1_file_size', 'severity_2', 'severity2_file_size', 
        'severity_3','severity3_file_size', 'severity_4', 'severity4_file_size',
        'Device Monitoring and Control', 'Device Monitoring and Control_file_size', 'Plug in Data Transfer', 
        'Plug in Data Transfer_file_size',
       'Post','Post_file_size', 'Print Control', 'Print Control_file_size',  'Uncategorized', 
        'Uncategorized_file_size', 'Upload', 'Upload_file_size',
       'Sum File Size']

cleaned = fullc[cols]
cleaned.sort_values('Total Violations', ascending=False)

# Cluster Analysis

In [226]:
fullc = cleaned.copy()

In [227]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd

feats = fullc.columns.tolist()
feats.remove('user')
fullc_no_user = fullc[feats]
scaler = StandardScaler()
fullc_scaled = scaler.fit_transform(fullc_no_user)

wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(fullc_scaled)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

optimal_clusters = 5
kmeans = KMeans(n_clusters=optimal_clusters, init='k-means++', random_state=42)
fullc['cluster'] = kmeans.fit_predict(fullc_scaled)



In [119]:
import seaborn as sns
sns.scatterplot(data=fullc, y='Total Violations', x='Unique Violation Days', hue='cluster', palette='viridis')
plt.title('Cluster Analysis Total Violations vs Unique Violation Days')
plt.show()

In [120]:
total_sev1 = fullc['severity_1'].sum()
total_sev2 = fullc['severity_2'].sum()
total_sev3 = fullc['severity_3'].sum()
total_sev4 = fullc['severity_4'].sum()
total_violations = total_sev1 + total_sev2 + total_sev3 + total_sev4
severity_levels = ['Sev1', 'Sev2', 'Sev3', 'Sev4', 'Total Violations']
counts = [total_sev1, total_sev2, total_sev3, total_sev4, total_violations]
sns.barplot(x=severity_levels, y=counts)
plt.xlabel('Severity Levels')
plt.ylabel('Total Count')
plt.title('Total Count of All Violations and Each Severity Level')

plt.show()

In [228]:
clusters = fullc['cluster'].unique()
for cluster in clusters:
    cluster_data = fullc[fullc['cluster'] == cluster]
    
    # Calculate the total violations per cluster
    total_violations = cluster_data[['severity_1', 'severity_2', 'severity_3', 'severity_4']].sum().sum()
    
    # Calculate the proportion of each severity level
    sev_proportions = cluster_data[['severity_1', 'severity_2', 'severity_3', 'severity_4']].sum() / total_violations
    
    # Plotting
    plt.figure(figsize=(10, 6))
    sev_proportions.plot(kind='bar')
    plt.title(f'Severity Distribution for Cluster {cluster}')
    plt.xlabel('Severity Level')
    plt.ylabel('Proportion')
    plt.show()

In [233]:
centroids = kmeans.cluster_centers_
centroids_df = pd.DataFrame(centroids, columns=feats)
centroids_df

,Unique Violation Days,Unique Rule Violations,Total Violations,severity_1,severity1_file_size,severity_2,severity2_file_size,severity_3,severity3_file_size,severity_4,severity4_file_size,Device Monitoring and Control,Device Monitoring and Control_file_size,Plug in Data Transfer,Plug in Data Transfer_file_size,Post,Post_file_size,Print Control,Print Control_file_size,Uncategorized,Uncategorized_file_size,Upload,Upload_file_size,Sum File Size
0,-0.002144,-0.004503,-0.009951,-0.007780,-0.015265,-0.001448,-0.014146,-0.013882,-0.010188,-0.021624,-0.019772,-0.012046,-0.015210,0.000065,0.000041,0.000091,-0.002127,-0.001568,-0.004339,-0.028512,-0.030886,-0.003463,-0.010937,-0.015451
1,-0.423669,1.417748,0.613876,-0.289776,-0.021244,6.305215,57.277382,-0.095957,-0.104259,-0.073460,-0.080628,-0.032107,-0.018337,-0.032897,-0.020730,0.065703,9.110524,-0.337113,-0.130976,-0.055171,-0.056495,15.026151,44.736846,0.273453
2,1.960343,3.108762,9.801987,10.142425,60.719330,-0.068949,-0.057810,0.559677,2.029902,0.193386,-0.076107,15.917247,60.732126,-0.032897,-0.020730,-0.061907,-0.079938,0.179888,0.015579,0.775689,1.595156,-0.157357,-0.109359,60.715294
3,1.564507,2.094154,1.918503,1.116304,0.048796,-0.068949,-0.057810,0.717029,4.326794,17.378262,15.946235,0.413595,-0.006607,-0.032897,-0.020730,-0.061907,-0.079938,1.359589,3.538596,14.512578,22.197724,-0.157357,-0.109359,0.127205
4,-0.738538,3.108762,20.001577,15.843598,0.429166,-0.068949,-0.057810,51.764683,17.402764,-0.073460,-0.080628,30.477528,0.469873,-0.032897,-0.020730,-0.061907,-0.079938,-0.337113,-0.130976,41.349362,11.649030,-0.157357,-0.109359,0.494594


# Cluster Centroid Features with Mean values > 1

In [230]:
num_clusters = centroids_df.shape
for i in range(num_clusters[0]):
    centroid = centroids_df.iloc[i].values 
    filtered_values = centroid[(centroid > 1) | (centroid < -1)]
    filtered_feats = np.array(feats)[(centroid > 1) | (centroid < -1)] 
    plt.figure(figsize=(10, 5))  
    plt.bar(np.arange(len(filtered_values)), filtered_values, color='skyblue') 
    plt.title(f'Cluster {i} Centroids (Filtered)')
    plt.xticks(np.arange(len(filtered_values)), filtered_feats, rotation=90)
    plt.ylabel('Feature Value')
    plt.show()

In [202]:
fullc

,user,Unique Violation Days,Unique Rule Violations,Total Violations,severity_1,severity1_file_size,severity_2,severity2_file_size,severity_3,severity3_file_size,severity_4,severity4_file_size,Device Monitoring and Control,Device Monitoring and Control_file_size,Plug in Data Transfer,Plug in Data Transfer_file_size,Post,Post_file_size,Print Control,Print Control_file_size,Uncategorized,Uncategorized_file_size,Upload,Upload_file_size,Sum File Size,cluster
0,A004,24,1,43,43,4639,0,0,0,0,0,0,43,4639,0,0,0,0,0,0,0,0,0,0,4639,0
1,A161,2,1,3,3,153847,0,0,0,0,0,0,3,153847,0,0,0,0,0,0,0,0,0,0,153847,0
2,A185133,17,1,66,66,327,0,0,0,0,0,0,0,0,0,0,0,0,66,327,0,0,0,0,327,0
3,A19739,4,1,7,7,327,0,0,0,0,0,0,0,0,0,0,0,0,7,327,0,0,0,0,327,0
4,A20061,1,1,1,0,0,0,0,1,296,0,0,0,0,0,0,0,0,0,0,0,0,1,296,296,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8052,zl97,1,1,1,1,32,0,0,0,0,0,0,0,0,0,0,0,0,1,32,0,0,0,0,32,0
8053,zp26,48,1,433,433,5522,0,0,0,0,0,0,0,0,0,0,0,0,433,5522,0,0,0,0,5522,0
8054,zp50,2,1,2,2,3,0,0,0,0,0,0,0,0,0,0,0,0,2,3,0,0,0,0,3,0
8055,zp86,2,1,2,2,684,0,0,0,0,0,0,0,0,0,0,0,0,2,684,0,0,0,0,684,0


# Assigning Categorical Risk to Clusters Value

In [234]:
from sklearn.metrics import silhouette_score
def assign_risk_score(row):
    if row['cluster'] == 0:
        return 'Lowest risk'  
    elif row['cluster'] == 1:
        return 'Moderate risk'  
    elif row['cluster'] == 2:
        return 'High risk'  
    elif row['cluster'] == 3:
        return 'Higher risk'  
    elif row['cluster'] == 4:
        return 'Highest risk'  
  

fullc['Risk_Score'] = fullc.apply(assign_risk_score, axis=1)

silhouette_avg = silhouette_score(fullc_scaled, fullc['cluster'])
print(f'Silhouette Score: {silhouette_avg}')

Silhouette Score: 0.9443012277880707


In [235]:
fullc['Risk_Score'].value_counts()

Risk_Score
Lowest risk      8041
Higher risk        10
Highest risk        2
High risk           2
Moderate risk       2
Name: count, dtype: int64

In [241]:
fullc

,user,Unique Violation Days,Unique Rule Violations,Total Violations,severity_1,severity1_file_size,severity_2,severity2_file_size,severity_3,severity3_file_size,severity_4,severity4_file_size,Device Monitoring and Control,Device Monitoring and Control_file_size,Plug in Data Transfer,Plug in Data Transfer_file_size,Post,Post_file_size,Print Control,Print Control_file_size,Uncategorized,Uncategorized_file_size,Upload,Upload_file_size,Sum File Size,cluster,Risk_Score
0,A004,24,1,43,43,4639,0,0,0,0,0,0,43,4639,0,0,0,0,0,0,0,0,0,0,4639,0,Lowest risk
1,A161,2,1,3,3,153847,0,0,0,0,0,0,3,153847,0,0,0,0,0,0,0,0,0,0,153847,0,Lowest risk
2,A185133,17,1,66,66,327,0,0,0,0,0,0,0,0,0,0,0,0,66,327,0,0,0,0,327,0,Lowest risk
3,A19739,4,1,7,7,327,0,0,0,0,0,0,0,0,0,0,0,0,7,327,0,0,0,0,327,0,Lowest risk
4,A20061,1,1,1,0,0,0,0,1,296,0,0,0,0,0,0,0,0,0,0,0,0,1,296,296,0,Lowest risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8052,zl97,1,1,1,1,32,0,0,0,0,0,0,0,0,0,0,0,0,1,32,0,0,0,0,32,0,Lowest risk
8053,zp26,48,1,433,433,5522,0,0,0,0,0,0,0,0,0,0,0,0,433,5522,0,0,0,0,5522,0,Lowest risk
8054,zp50,2,1,2,2,3,0,0,0,0,0,0,0,0,0,0,0,0,2,3,0,0,0,0,3,0,Lowest risk
8055,zp86,2,1,2,2,684,0,0,0,0,0,0,0,0,0,0,0,0,2,684,0,0,0,0,684,0,Lowest risk
